<img src='https://www.unifor.br/o/unifor-theme/images/unifor-logo-horizontal.svg' width="250px">

# Ciência de Dados
Prof. Caio Ponte

Universidade de Fortaleza

## Dataset: Olist

### Descrição
O **Brazilian E-Commerce Public Dataset by Olist** é um conjunto de dados transacionais de um marketplace brasileiro, contendo informações de aproximadamente 100 mil pedidos realizados entre 2016 e 2018. O Olist é uma startup que conecta pequenos lojistas a grandes marketplaces, facilitando a gestão e a venda de produtos online.

Referência: [Link do Dataset](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce/)

### Análises Realizadas

Este notebook responde a 4 perguntas específicas sobre o dataset:

1. **Qual o percentual de pedidos entregues após a data estimada pela Olist?**
2. **Qual o método de pagamento mais utilizado em pedidos acima de R$ 150,00?**
3. **Qual é a relação entre o tempo de entrega e a nota de avaliação do cliente?**
4. **Quais são as 5 categorias de produtos mais vendidas e qual a receita total gerada por cada uma?**

## Configuração Inicial

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

## Carregamento dos Dados

In [ ]:
# Carregar dados usando o data_loader
from data_loader import load_data

print("Carregando dados do Olist...")
datasets = load_data()

print(f"Datasets disponíveis: {list(datasets.keys())}")
print(f"Número de pedidos: {len(datasets['orders'])}")

## Pergunta 1: Percentual de Pedidos Entregues Após a Data Estimada

### Metodologia
- Filtrar pedidos entregues com datas válidas
- Calcular atraso comparando data de entrega real vs estimada
- Determinar percentual de pedidos atrasados

In [ ]:
def pergunta_1_entregas_atrasadas(datasets):
    """
    Pergunta 1: Qual o percentual de pedidos entregues após a data estimada pela Olist?
    """
    print("=" * 70)
    print("PERGUNTA 1: Percentual de pedidos entregues após a data estimada")
    print("=" * 70)
    
    orders = datasets['orders'].copy()
    
    # Filtrar pedidos entregues e com datas válidas
    delivered_orders = orders[
        (orders['order_status'] == 'delivered') &
        (orders['order_delivered_customer_date'].notna()) &
        (orders['order_estimated_delivery_date'].notna())
    ].copy()
    
    if len(delivered_orders) == 0:
        print("Não há dados suficientes para análise de entregas.")
        return None
    
    # Converter para datetime
    delivered_orders['order_delivered_customer_date'] = pd.to_datetime(
        delivered_orders['order_delivered_customer_date']
    )
    delivered_orders['order_estimated_delivery_date'] = pd.to_datetime(
        delivered_orders['order_estimated_delivery_date']
    )
    
    # Calcular atraso
    delivered_orders['atraso_dias'] = (
        delivered_orders['order_delivered_customer_date'] - 
        delivered_orders['order_estimated_delivery_date']
    ).dt.days
    
    # Calcular estatísticas
    total_entregas = len(delivered_orders)
    entregas_atrasadas = len(delivered_orders[delivered_orders['atraso_dias'] > 0])
    percentual_atraso = (entregas_atrasadas / total_entregas) * 100
    
    # Apresentar resultados
    print(f"Total de entregas analisadas: {total_entregas:,}")
    print(f"Entregas atrasadas: {entregas_atrasadas:,}")
    print(f"Percentual de entregas atrasadas: {percentual_atraso:.2f}%")
    print(f"Atraso médio (apenas pedidos atrasados): {delivered_orders[delivered_orders['atraso_dias'] > 0]['atraso_dias'].mean():.1f} dias")
    
    # Criar visualizações
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Análise de Entregas - Prazo vs Realizado', fontsize=16)
    
    # 1. Distribuição de atrasos
    axes[0, 0].hist(delivered_orders['atraso_dias'], bins=50, alpha=0.7, color='steelblue')
    axes[0, 0].axvline(0, color='red', linestyle='--', label='Prazo estimado')
    axes[0, 0].set_xlabel('Atraso (dias)')
    axes[0, 0].set_ylabel('Frequência')
    axes[0, 0].set_title('Distribuição dos Atrasos')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Pizza - No prazo vs Atrasado
    labels = ['No Prazo', 'Atrasado']
    sizes = [total_entregas - entregas_atrasadas, entregas_atrasadas]
    colors = ['lightgreen', 'lightcoral']
    
    axes[0, 1].pie(sizes, labels=labels, autopct='%1.2f%%', colors=colors, startangle=90)
    axes[0, 1].set_title('Distribuição: Entregas no Prazo vs Atrasadas')
    
    # 3. Boxplot dos atrasos
    delivered_orders['status_entrega'] = delivered_orders['atraso_dias'].apply(
        lambda x: 'No Prazo' if x <= 0 else 'Atrasado'
    )
    sns.boxplot(data=delivered_orders, x='status_entrega', y='atraso_dias', ax=axes[1, 0])
    axes[1, 0].set_title('Boxplot: Distribuição dos Atrasos por Status')
    axes[1, 0].set_ylabel('Atraso (dias)')
    
    # 4. Estatísticas resumidas
    stats_text = f"""
    ESTATÍSTICAS RESUMIDAS
    
    Total de entregas: {total_entregas:,}
    Entregas no prazo: {total_entregas - entregas_atrasadas:,}
    Entregas atrasadas: {entregas_atrasadas:,}
    
    Percentual de atraso: {percentual_atraso:.2f}%
    
    Atraso médio: {delivered_orders['atraso_dias'].mean():.1f} dias
    Atraso mediano: {delivered_orders['atraso_dias'].median():.1f} dias
    
    Maior atraso: {delivered_orders['atraso_dias'].max():.0f} dias
    Maior antecipação: {abs(delivered_orders['atraso_dias'].min()):.0f} dias
    """
    axes[1, 1].text(0.05, 0.95, stats_text, transform=axes[1, 1].transAxes, 
                    verticalalignment='top', fontsize=10, 
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.5))
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return {
        'total_entregas': total_entregas,
        'entregas_atrasadas': entregas_atrasadas,
        'percentual_atraso': percentual_atraso,
        'atraso_medio': delivered_orders['atraso_dias'].mean()
    }

# Executar análise
resultado_pergunta1 = pergunta_1_entregas_atrasadas(datasets)

## Pergunta 2: Método de Pagamento Mais Utilizado em Pedidos > R$ 150,00

### Metodologia
- Filtrar pagamentos com valor acima de R$ 150,00
- Agrupar por tipo de pagamento
- Calcular frequência e percentuais
- Identificar método mais utilizado

In [ ]:
def pergunta_2_metodo_pagamento(datasets):
    """
    Pergunta 2: Qual o método de pagamento mais utilizado em pedidos acima de R$ 150,00?
    """
    print("=" * 70)
    print("PERGUNTA 2: Método de pagamento mais usado em pedidos > R$ 150,00")
    print("=" * 70)
    
    payments = datasets['order_payments'].copy()
    
    # Filtrar pedidos acima de R$ 150,00
    payments_above_150 = payments[payments['payment_value'] > 150.0]
    
    if len(payments_above_150) == 0:
        print("Não há pedidos acima de R$ 150,00.")
        return None
    
    # Agrupar por tipo de pagamento
    payment_counts = payments_above_150.groupby('payment_type').agg({
        'order_id': 'count',
        'payment_value': ['sum', 'mean']
    }).round(2)
    
    payment_counts.columns = ['quantidade_pedidos', 'valor_total', 'valor_medio']
    payment_counts = payment_counts.sort_values('quantidade_pedidos', ascending=False)
    
    # Calcular percentuais
    total_pedidos = payment_counts['quantidade_pedidos'].sum()
    payment_counts['percentual'] = (payment_counts['quantidade_pedidos'] / total_pedidos * 100).round(2)
    
    # Método mais utilizado
    metodo_mais_usado = payment_counts.index[0]
    
    # Apresentar resultados
    print(f"Total de pedidos > R$ 150,00: {total_pedidos:,}")
    print(f"Método de pagamento mais utilizado: {metodo_mais_usado}")
    print(f"Participação: {payment_counts.loc[metodo_mais_usado, 'percentual']:.2f}%")
    print("\nRanking completo:")
    print("-" * 60)
    
    for i, (metodo, dados) in enumerate(payment_counts.iterrows(), 1):
        print(f"{i}. {metodo}")
        print(f"   Pedidos: {dados['quantidade_pedidos']:,} ({dados['percentual']:.2f}%)")
        print(f"   Valor médio: R$ {dados['valor_medio']:.2f}")
        print()
    
    # Criar visualizações
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Análise: Métodos de Pagamento (Pedidos > R$ 150)', fontsize=16)
    
    # 1. Gráfico de barras - Quantidade de pedidos
    colors = plt.cm.Set3(np.arange(len(payment_counts)))
    bars1 = axes[0, 0].bar(range(len(payment_counts)), payment_counts['quantidade_pedidos'], 
                          color=colors, alpha=0.8)
    axes[0, 0].set_xlabel('Método de Pagamento')
    axes[0, 0].set_ylabel('Quantidade de Pedidos')
    axes[0, 0].set_title('Quantidade de Pedidos por Método')
    axes[0, 0].set_xticks(range(len(payment_counts)))
    axes[0, 0].set_xticklabels(payment_counts.index, rotation=45, ha='right')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Adicionar valores nas barras
    for i, bar in enumerate(bars1):
        height = bar.get_height()
        axes[0, 0].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                       f'{int(height):,}', ha='center', va='bottom', fontsize=9)
    
    # 2. Gráfico de pizza - Distribuição percentual
    axes[0, 1].pie(payment_counts['quantidade_pedidos'], labels=payment_counts.index, 
                   autopct='%1.2f%%', colors=colors, startangle=90)
    axes[0, 1].set_title('Distribuição dos Métodos de Pagamento')
    
    # 3. Gráfico de barras - Valor médio
    bars2 = axes[1, 0].bar(range(len(payment_counts)), payment_counts['valor_medio'], 
                          color=colors, alpha=0.8)
    axes[1, 0].set_xlabel('Método de Pagamento')
    axes[1, 0].set_ylabel('Valor Médio (R$)')
    axes[1, 0].set_title('Valor Médio por Método')
    axes[1, 0].set_xticks(range(len(payment_counts)))
    axes[1, 0].set_xticklabels(payment_counts.index, rotation=45, ha='right')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Adicionar valores nas barras
    for i, bar in enumerate(bars2):
        height = bar.get_height()
        axes[1, 0].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                       f'R$ {height:.0f}', ha='center', va='bottom', fontsize=9)
    
    # 4. Resumo estatístico
    stats_text = f"""
    RESUMO ESTATÍSTICO
    
    Total de pedidos > R$ 150: {total_pedidos:,}
    
    Método mais usado: {metodo_mais_usado}
    Participação: {payment_counts.loc[metodo_mais_usado, 'percentual']:.2f}%
    
    Valor total transacionado: 
    R$ {payment_counts['valor_total'].sum():,.2f}
    
    Valor médio geral: 
    R$ {payments_above_150['payment_value'].mean():.2f}
    """
    axes[1, 1].text(0.05, 0.95, stats_text, transform=axes[1, 1].transAxes, 
                    verticalalignment='top', fontsize=10,
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.5))
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return {
        'metodo_mais_usado': metodo_mais_usado,
        'ranking_completo': payment_counts,
        'total_pedidos': total_pedidos
    }

# Executar análise
resultado_pergunta2 = pergunta_2_metodo_pagamento(datasets)

## Pergunta 3: Relação entre Tempo de Entrega e Nota de Avaliação

### Metodologia
- Merge entre tabelas orders e order_reviews
- Calcular tempo de entrega em dias
- Remover outliers extremos
- Calcular correlação de Pearson
- Analisar estatísticas por nota de avaliação

In [ ]:
def pergunta_3_tempo_entrega_avaliacao(datasets):
    """
    Pergunta 3: Qual é a relação entre o tempo de entrega e a nota de avaliação do cliente?
    """
    print("=" * 70)
    print("PERGUNTA 3: Relação entre tempo de entrega e avaliação do cliente")
    print("=" * 70)
    
    orders = datasets['orders'].copy()
    reviews = datasets['order_reviews'].copy()
    
    # Converter colunas de data
    orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
    orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
    
    # Merge entre pedidos e avaliações
    orders_reviews = orders.merge(reviews, on='order_id', how='inner')
    
    # Filtrar pedidos entregues com dados válidos
    delivered_reviews = orders_reviews[
        (orders_reviews['order_status'] == 'delivered') &
        (orders_reviews['order_delivered_customer_date'].notna()) &
        (orders_reviews['order_purchase_timestamp'].notna()) &
        (orders_reviews['review_score'].notna())
    ].copy()
    
    if len(delivered_reviews) == 0:
        print("Não há dados suficientes para análise.")
        return None
    
    # Calcular tempo de entrega em dias
    delivered_reviews['tempo_entrega_dias'] = (
        delivered_reviews['order_delivered_customer_date'] - 
        delivered_reviews['order_purchase_timestamp']
    ).dt.days
    
    # Remover outliers extremos
    before_filter = len(delivered_reviews)
    delivered_reviews = delivered_reviews[
        (delivered_reviews['tempo_entrega_dias'] >= 0) & 
        (delivered_reviews['tempo_entrega_dias'] <= 100)
    ]
    
    print(f"Registros após remoção de outliers: {len(delivered_reviews)} (removidos: {before_filter - len(delivered_reviews)})")
    
    if len(delivered_reviews) == 0:
        print("Não há dados válidos após remoção de outliers.")
        return None
    
    # Análise estatística por nota
    stats_by_score = delivered_reviews.groupby('review_score')['tempo_entrega_dias'].agg([
        'count', 'mean', 'median', 'std'
    ]).round(2)
    
    # Calcular correlação
    correlation = delivered_reviews['tempo_entrega_dias'].corr(delivered_reviews['review_score'])
    
    # Apresentar resultados
    print(f"Total de avaliações analisadas: {len(delivered_reviews)}")
    print(f"Tempo médio de entrega: {delivered_reviews['tempo_entrega_dias'].mean():.1f} dias")
    print(f"Correlação de Pearson: {correlation:.4f}")
    
    # Interpretar correlação
    if abs(correlation) < 0.1:
        interpretacao = "muito fraca"
    elif abs(correlation) < 0.3:
        interpretacao = "fraca"
    elif abs(correlation) < 0.5:
        interpretacao = "moderada"
    elif abs(correlation) < 0.7:
        interpretacao = "forte"
    else:
        interpretacao = "muito forte"
    
    direcao = "positiva" if correlation > 0 else "negativa"
    print(f"Interpretação: Correlação {interpretacao} {direcao}")
    
    print("\nEstatísticas por nota de avaliação:")
    print(stats_by_score)
    
    # Criar visualizações
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Análise: Tempo de Entrega vs Avaliação do Cliente', fontsize=16)
    
    # 1. Scatter plot
    axes[0, 0].scatter(delivered_reviews['tempo_entrega_dias'], 
                      delivered_reviews['review_score'], alpha=0.6, s=30)
    axes[0, 0].set_xlabel('Tempo de Entrega (dias)')
    axes[0, 0].set_ylabel('Nota da Avaliação')
    axes[0, 0].set_title(f'Scatter Plot (Correlação: {correlation:.4f})')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Linha de tendência
    z = np.polyfit(delivered_reviews['tempo_entrega_dias'], 
                   delivered_reviews['review_score'], 1)
    p = np.poly1d(z)
    axes[0, 0].plot(delivered_reviews['tempo_entrega_dias'], 
                    p(delivered_reviews['tempo_entrega_dias']), "r--", alpha=0.8)
    
    # 2. Boxplot por nota
    sns.boxplot(data=delivered_reviews, x='review_score', y='tempo_entrega_dias', ax=axes[0, 1])
    axes[0, 1].set_xlabel('Nota da Avaliação')
    axes[0, 1].set_ylabel('Tempo de Entrega (dias)')
    axes[0, 1].set_title('Distribuição do Tempo por Nota')
    
    # 3. Histograma dos tempos de entrega
    axes[1, 0].hist(delivered_reviews['tempo_entrega_dias'], bins=30, 
                    alpha=0.7, color='steelblue', edgecolor='black')
    axes[1, 0].axvline(delivered_reviews['tempo_entrega_dias'].mean(), 
                       color='red', linestyle='--', label='Média')
    axes[1, 0].set_xlabel('Tempo de Entrega (dias)')
    axes[1, 0].set_ylabel('Frequência')
    axes[1, 0].set_title('Distribuição dos Tempos de Entrega')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Tempo médio por nota
    avg_time_by_score = delivered_reviews.groupby('review_score')['tempo_entrega_dias'].mean()
    bars = axes[1, 1].bar(avg_time_by_score.index, avg_time_by_score.values, 
                         color='lightcoral', alpha=0.8, edgecolor='black')
    axes[1, 1].set_xlabel('Nota da Avaliação')
    axes[1, 1].set_ylabel('Tempo Médio de Entrega (dias)')
    axes[1, 1].set_title('Tempo Médio de Entrega por Nota')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Adicionar valores nas barras
    for bar in bars:
        height = bar.get_height()
        axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                        f'{height:.1f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'correlacao': correlation,
        'interpretacao': f"{interpretacao} {direcao}",
        'stats_por_nota': stats_by_score,
        'tempo_medio_geral': delivered_reviews['tempo_entrega_dias'].mean()
    }

# Executar análise
resultado_pergunta3 = pergunta_3_tempo_entrega_avaliacao(datasets)

## Pergunta 4: Top 5 Categorias de Produtos Mais Vendidas

### Metodologia
- Merge entre order_items e products
- Agrupar por categoria de produto
- Calcular quantidade vendida e receita total
- Ordenar por quantidade e selecionar top 5
- Traduzir nomes quando disponível

In [ ]:
def pergunta_4_top_categorias(datasets):
    """
    Pergunta 4: Quais são as 5 categorias de produtos mais vendidas e qual a receita total gerada por cada uma?
    """
    print("=" * 70)
    print("PERGUNTA 4: Top 5 categorias de produtos mais vendidas")
    print("=" * 70)
    
    order_items = datasets['order_items'].copy()
    products = datasets['products'].copy()
    
    # Merge entre itens e produtos
    items_products = order_items.merge(products, on='product_id', how='inner')
    
    if len(items_products) == 0:
        print("Não foi possível fazer o merge entre itens e produtos.")
        return None
    
    # Agrupar por categoria
    category_stats = items_products.groupby('product_category_name').agg({
        'order_id': 'count',  # Quantidade vendida
        'price': 'sum'        # Receita total
    }).round(2)
    
    category_stats.columns = ['quantidade_vendida', 'receita_total']
    category_stats = category_stats.sort_values('quantidade_vendida', ascending=False)
    
    # Top 5 categorias
    top_5_categories = category_stats.head(5)
    
    # Calcular ticket médio
    top_5_categories['ticket_medio'] = (
        top_5_categories['receita_total'] / top_5_categories['quantidade_vendida']
    ).round(2)
    
    # Traduzir nomes se disponível
    top_5_display = top_5_categories.copy()
    if 'category_translation' in datasets:
        translation = datasets['category_translation'].set_index('product_category_name')
        for idx in top_5_categories.index:
            if idx in translation.index:
                new_name = translation.loc[idx, 'product_category_name_english']
                top_5_display = top_5_display.rename(index={idx: new_name})
    
    # Apresentar resultados
    print("Top 5 categorias de produtos mais vendidas:")
    print("-" * 50)
    
    for i, (categoria, dados) in enumerate(top_5_display.iterrows(), 1):
        print(f"{i}. {categoria}")
        print(f"   Quantidade vendida: {dados['quantidade_vendida']:,}")
        print(f"   Receita total: R$ {dados['receita_total']:,.2f}")
        print(f"   Ticket médio: R$ {dados['ticket_medio']:.2f}")
        print()
    
    total_receita_top5 = top_5_categories['receita_total'].sum()
    total_vendas_top5 = top_5_categories['quantidade_vendida'].sum()
    print(f"Receita total das top 5: R$ {total_receita_top5:,.2f}")
    print(f"Total de vendas das top 5: {total_vendas_top5:,} itens")
    
    # Criar visualizações
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Análise: Top 5 Categorias de Produtos Mais Vendidas', fontsize=16)
    
    # 1. Gráfico de barras - Quantidade vendida
    colors = plt.cm.Set3(np.arange(len(top_5_display)))
    bars1 = axes[0, 0].bar(range(len(top_5_display)), top_5_display['quantidade_vendida'], 
                          color=colors, alpha=0.8, edgecolor='black')
    axes[0, 0].set_xlabel('Categorias')
    axes[0, 0].set_ylabel('Quantidade Vendida')
    axes[0, 0].set_title('Quantidade Vendida por Categoria')
    axes[0, 0].set_xticks(range(len(top_5_display)))
    axes[0, 0].set_xticklabels([cat[:15] + '...' if len(cat) > 15 else cat 
                               for cat in top_5_display.index], rotation=45, ha='right')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Adicionar valores nas barras
    for i, bar in enumerate(bars1):
        height = bar.get_height()
        axes[0, 0].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                       f'{int(height):,}', ha='center', va='bottom', fontsize=9)
    
    # 2. Gráfico de barras - Receita total
    bars2 = axes[0, 1].bar(range(len(top_5_display)), top_5_display['receita_total'], 
                          color=colors, alpha=0.8, edgecolor='black')
    axes[0, 1].set_xlabel('Categorias')
    axes[0, 1].set_ylabel('Receita Total (R$)')
    axes[0, 1].set_title('Receita Total por Categoria')
    axes[0, 1].set_xticks(range(len(top_5_display)))
    axes[0, 1].set_xticklabels([cat[:15] + '...' if len(cat) > 15 else cat 
                               for cat in top_5_display.index], rotation=45, ha='right')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Adicionar valores nas barras
    for i, bar in enumerate(bars2):
        height = bar.get_height()
        axes[0, 1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                       f'R$ {height:,.0f}', ha='center', va='bottom', fontsize=8)
    
    # 3. Gráfico de pizza - Distribuição da receita
    axes[1, 0].pie(top_5_display['receita_total'], labels=top_5_display.index, 
                   autopct='%1.1f%%', colors=colors, startangle=90)
    axes[1, 0].set_title('Distribuição da Receita (Top 5)')
    
    # 4. Ticket médio
    bars3 = axes[1, 1].bar(range(len(top_5_display)), top_5_display['ticket_medio'], 
                          color=colors, alpha=0.8, edgecolor='black')
    axes[1, 1].set_xlabel('Categorias')
    axes[1, 1].set_ylabel('Ticket Médio (R$)')
    axes[1, 1].set_title('Ticket Médio por Categoria')
    axes[1, 1].set_xticks(range(len(top_5_display)))
    axes[1, 1].set_xticklabels([cat[:15] + '...' if len(cat) > 15 else cat 
                               for cat in top_5_display.index], rotation=45, ha='right')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Adicionar valores nas barras
    for i, bar in enumerate(bars3):
        height = bar.get_height()
        axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                       f'R$ {height:.2f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'top_5_categories': top_5_categories,
        'top_5_categories_display': top_5_display,
        'total_receita_top5': total_receita_top5,
        'total_vendas_top5': total_vendas_top5
    }

# Executar análise
resultado_pergunta4 = pergunta_4_top_categorias(datasets)

## Resumo Final das Análises

### Principais Descobertas

In [ ]:
def gerar_relatorio_final(resultado_p1, resultado_p2, resultado_p3, resultado_p4):
    """
    Gera um relatório final com os principais resultados
    """
    print("=" * 80)
    print("RELATÓRIO FINAL - ANÁLISE OLIST")
    print("=" * 80)
    
    # Pergunta 1
    if resultado_p1:
        print(f"\n1. ENTREGAS APÓS PRAZO ESTIMADO:")
        print(f"   • Percentual de entregas atrasadas: {resultado_p1['percentual_atraso']:.2f}%")
        print(f"   • Total de entregas analisadas: {resultado_p1['total_entregas']:,}")
    
    # Pergunta 2
    if resultado_p2:
        print(f"\n2. MÉTODO DE PAGAMENTO (PEDIDOS > R$ 150):")
        print(f"   • Método mais usado: {resultado_p2['metodo_mais_usado']}")
        top_metodo_pct = resultado_p2['ranking_completo'].iloc[0]['percentual']
        print(f"   • Participação: {top_metodo_pct:.2f}%")
        print(f"   • Total de pedidos analisados: {resultado_p2['total_pedidos']:,}")
    
    # Pergunta 3
    if resultado_p3:
        print(f"\n3. TEMPO DE ENTREGA VS AVALIAÇÃO:")
        print(f"   • Correlação: {resultado_p3['correlacao']:.4f} ({resultado_p3['interpretacao']})")
        print(f"   • Tempo médio de entrega: {resultado_p3['tempo_medio_geral']:.1f} dias")
    
    # Pergunta 4
    if resultado_p4:
        print(f"\n4. TOP 5 CATEGORIAS MAIS VENDIDAS:")
        top_5 = resultado_p4['top_5_categories_display']
        for i, (categoria, dados) in enumerate(top_5.iterrows(), 1):
            print(f"   {i}. {categoria}: {int(dados['quantidade_vendida']):,} itens | R$ {dados['receita_total']:,.2f}")
        print(f"   • Receita total (top 5): R$ {resultado_p4['total_receita_top5']:,.2f}")
    
    print(f"\n{'='*80}")
    print("Análise concluída com sucesso!")
    print(f"{'='*80}")

# Gerar relatório final
gerar_relatorio_final(resultado_pergunta1, resultado_pergunta2, resultado_pergunta3, resultado_pergunta4)

## Conclusões

### Metodologias Aplicadas

**Pergunta 1 - Entregas Atrasadas:**
- Filtragem de pedidos entregues com datas válidas
- Cálculo de diferença entre data real e estimada
- Análise percentual e estatística descritiva

**Pergunta 2 - Métodos de Pagamento:**
- Filtragem por valor de pedido (> R$ 150)
- Agrupamento por tipo de pagamento
- Cálculo de frequências e percentuais

**Pergunta 3 - Tempo vs Avaliação:**
- Merge de datasets com join interno
- Cálculo de tempo de entrega em dias
- Remoção de outliers extremos
- Análise de correlação de Pearson

**Pergunta 4 - Top Categorias:**
- Merge entre itens de pedidos e produtos
- Agrupamento por categoria
- Ordenação por quantidade vendida
- Cálculo de métricas de receita

### Insights de Negócio

As análises fornecem insights valiosos para:
- **Gestão de expectativas**: Monitoramento de prazos de entrega
- **Estratégia de pagamentos**: Foco nos métodos preferidos pelos clientes
- **Qualidade do serviço**: Relação entre tempo de entrega e satisfação
- **Mix de produtos**: Identificação de categorias de maior performance

Estes dados podem orientar decisões estratégicas e operacionais para melhoria contínua do marketplace.